# ETL

En este notebook, se realizarán las transformaciones necesarias para carga de los datos en MySQL y luego uso de los mismos en análisis. Para ello es necesario hacer cierto análisis de la información disponible, por lo que mucho de lo que aquí se ve está también en el [EDA](EDA.ipynb)

Importamos las librerías necesarias

In [18]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

Iniciamos la transformación de datos del archivo homicidios.xlsx, la hoja de hechos. <br>
El dataset puede descargarse [aquí](https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/homicidios.xlsx)

In [19]:
df_hechos = pd.read_excel('Datasets/homicidios.xlsx', sheet_name='HECHOS')
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [20]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Veamos valores duplicados:

In [21]:
df_hechos.duplicated().sum()

0

Ahora veamos valores nulos:

In [22]:
df_hechos.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Vemos que hay nulos en las columnas Calle y Altura. Sin embargo, puede haber valores nulos que en el dataframe aparecen como string.
Reemplazemos los posibles valores faltantes por np.Nan. Para ello inicialmente cambiamos todos los valores string a letra minúscula

In [23]:
df_hechos = df_hechos.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [24]:
faltantes = ['', ' ', '.', '...', 'sd', 'sd-sd', 'point (. .)']
df_hechos.replace(faltantes, np.nan, inplace=True)

In [25]:
df_hechos.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                 14
pos x                     12
pos y                     12
PARTICIPANTES              5
VICTIMA                    9
ACUSADO                   23
dtype: int64

Ahora sí vemos, en los resultados anteriores, la cantidad de valores faltantes reales.

Configuramos pandas para que muestre todas las columnas en Jupyter Notebook

In [26]:
pd.set_option('display.max_columns', None)

Dado que utilizaremos los datos geográficos con las columnas pos x (longitud) y pos y (latitud), los faltantes en las columnas Calle, Altura, Cruce, Dirección Normalizada, XY (CABA) y LUGAR_DEL_HECHO podemos obviarlos y luego de analizar los faltantes de latitud y longitud, proceder a eliminar las columnas.

Veamos si podemos imputar los faltantes de las columnas pos x y pos y

In [27]:
df_hechos[df_hechos['pos x'].isna()][['LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Dirección Normalizada']]

,LUGAR_DEL_HECHO,Calle,Altura,Dirección Normalizada
38,autopista lugones pk 10000,"lugones, leopoldo av.",NaN,NaN
106,au buenos aires - la plata km. 4,autopista buenos aires - la plata,NaN,NaN
119,NaN,NaN,NaN,NaN
139,autopista 25 de mayo,autopista 25 de mayo,NaN,autopista 25 de mayo
176,av. leopoldo lugones pkm 6900,"lugones, leopoldo av.",NaN,"lugones, leopoldo av."
180,au perito moreno y ramal enlace au1/au6,autopista perito moreno,NaN,NaN
181,au dellepiane 2400,autopista dellepiane luis tte. gral.,NaN,NaN
256,au arturo frondizi pkm 3100,autopista 1 sur presidente arturo frondizi,NaN,autopista 1 sur presidente arturo frondizi
313,autopista lugones km 4.7,"lugones, leopoldo av.",NaN,NaN
546,"lugones, leopoldo av. km 6,1","lugones, leopoldo av.",NaN,NaN


La mayoría de los datos los podemos obtener investigando sobre los puntos kilometricos (pk o pkm) cada autopista o numeración de avenida. A continuación un diccionario donde cada índice se relaciona con pos x (longitud) y pos y (latitud). Si bien los datos obtenidos son aproximados, son útiles a los fines de nuestro análisis para posicionarlos en un mapa. Esta información la obtenemos en [este enlace](https://mapa.buenosaires.gob.ar/comollego/?lat=-34.620000&lng=-58.440000&zl=12&modo=transporte)
Sumado a la investigación de cómo está numerada cada calle, avenida y autopista.

In [28]:
df_hechos.rename(columns= {'pos x': 'long', 'pos y': 'lat'}, inplace=True)

In [29]:
# mapea indice a [lat, long]
mapeo_pos = {
    38: [-34.546421, -58.442395],
    106: [-34.624967, -58.362637],
    176: [-34.567216, -58.408535],
    180: [-34.649657, -58.465934],
    181: [-34.643183, -58.462656],  # este punto se hace coincidir con comuna 7, rotonda dellepiane / au 25 de mayo
    256: [-34.630948, -58.378601],
    313: [-34.559287, -58.420465],
    546: [-34.565585, -58.411064],
    559: [-34.663397, -58.493317],
    621: [-34.628112, -58.357098]   
}

En el siguiente bucle lo que hacemos es asignar el valor del diccionario (ya sea latitud o longitud) a la posición correspondiente en el dataframe.

In [30]:
indices = mapeo_pos.keys()
for i in indices:
    df_hechos.loc[i, 'lat'] = mapeo_pos[i][0]
    df_hechos.loc[i, 'long'] = mapeo_pos[i][1]

Veamos si la imputación fue correcta:

In [31]:
df_hechos.iloc[list(indices), :]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),long,lat,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,autopista lugones pk 10000,autopista,"lugones, leopoldo av.",NaN,NaN,NaN,13,NaN,-58.442395,-34.546421,moto-sd,moto,NaN
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0.0,au buenos aires - la plata km. 4,autopista,autopista buenos aires - la plata,NaN,NaN,NaN,4,NaN,-58.362637,-34.624967,moto-cargas,moto,cargas
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9.0,av. leopoldo lugones pkm 6900,gral paz,"lugones, leopoldo av.",NaN,NaN,"lugones, leopoldo av.",14,NaN,-58.408535,-34.567216,moto-cargas,moto,cargas
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11.0,au perito moreno y ramal enlace au1/au6,autopista,autopista perito moreno,NaN,NaN,NaN,9,NaN,-58.465934,-34.649657,moto-cargas,moto,cargas
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3.0,au dellepiane 2400,autopista,autopista dellepiane luis tte. gral.,NaN,NaN,NaN,7,NaN,-58.462656,-34.643183,auto-auto,auto,auto
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23.0,au arturo frondizi pkm 3100,autopista,autopista 1 sur presidente arturo frondizi,NaN,NaN,autopista 1 sur presidente arturo frondizi,4,NaN,-58.378601,-34.630948,moto-pasajeros,moto,pasajeros
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22.0,autopista lugones km 4.7,autopista,"lugones, leopoldo av.",NaN,NaN,NaN,14,NaN,-58.420465,-34.559287,peaton-auto,peaton,auto
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6.0,"lugones, leopoldo av. km 6,1",autopista,"lugones, leopoldo av.",NaN,NaN,NaN,14,NaN,-58.411064,-34.565585,moto-objeto fijo,moto,objeto fijo
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19.0,murguiondo 2700,calle,murguiondo,NaN,NaN,murguiondo,9,NaN,-58.493317,-34.663397,peaton-cargas,peaton,cargas
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9.0,"au buenos aires la plata km 4,5",autopista,autopista buenos aires - la plata,NaN,NaN,NaN,4,NaN,-58.357098,-34.628112,moto-cargas,moto,cargas


El id 2017-0042 tiene incorrecto el dato 'tipo_de_calle', que debe ser autopista. A continuación lo corregimos.

In [32]:
df_hechos.loc[176, 'TIPO_DE_CALLE'] = 'autopista'

Nos quedan sin imputar las posiciones de los índices 119 y 139. Veamos estas dos filas:

In [33]:
df_hechos.iloc[[119, 139], :]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),long,lat,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,NaN,calle,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,peaton-sd,peaton,NaN
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0.0,autopista 25 de mayo,autopista,autopista 25 de mayo,NaN,NaN,autopista 25 de mayo,0,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que faltan los datos más importantes, además el valor de Comuna 0 también corresponde a un dato faltante, ya que las comunas en CABA están numeradas comenzando en 1. Por lo tanto estos dos registros no servirán para el análisis y procedemos a eliminarlos.

In [34]:
df_hechos.drop(index=[119, 139], inplace=True)

Veamos si hay más registros con valor 0 en Comuna:

In [35]:
df_hechos[df_hechos['COMUNA'].isin([0])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),long,lat,PARTICIPANTES,VICTIMA,ACUSADO


No hay más Faltantes en la columna Comuna. En esta [URL](https://buenosaires.gob.ar/jefaturadegabinete/atencion-ciudadana-y-gestion-comunal/gestion-comunal/comunas) podrás encontrar información sobre las comunas en CABA. La Ciudad de Buenos Aires se encuentra organizada en 15 Comunas. Por lo tanto el máximo en esta columna debe ser 15. Veamos los valores de la columna COMUNA

In [36]:
sorted(df_hechos['COMUNA'].unique().tolist())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

De esta manera, verificamos que los datos de la columna comuna están en el rango esperado.

Ahora sí, procedemos a eliminar las columnas que no utilizaremos 

In [37]:
df_hechos = df_hechos.drop(columns=['Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 'LUGAR_DEL_HECHO'])

Veamos nuevamente la suma de los nulos por columna:

In [38]:
df_hechos.isna().sum()

ID                0
N_VICTIMAS        0
FECHA             0
AAAA              0
MM                0
DD                0
HORA              1
HH                1
TIPO_DE_CALLE     0
COMUNA            0
long              0
lat               0
PARTICIPANTES     4
VICTIMA           8
ACUSADO          21
dtype: int64

Veamos los faltantes en la columna ACUSADO

In [39]:
df_hechos[df_hechos['ACUSADO'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,long,lat,PARTICIPANTES,VICTIMA,ACUSADO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,avenida,8,-58.46503904,-34.68092974,moto-sd,moto,NaN
32,2016-0045,1,2016-04-11,2016,4,11,10:25:00,10.0,gral paz,13,-58.46743471,-34.53476874,moto-sd,moto,NaN
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0.0,autopista,4,-58.37714647568196,-34.63657525428238,NaN,NaN,NaN
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,autopista,13,-58.442395,-34.546421,moto-sd,moto,NaN
57,2016-0080,1,2016-06-18,2016,6,18,06:42:00,6.0,gral paz,9,-58.52927982,-34.63931752,auto-sd,auto,NaN
76,2016-0101,1,2016-08-07,2016,8,7,19:42:00,19.0,gral paz,9,-58.52932872,-34.65014993,NaN,NaN,NaN
83,2016-0110,1,2016-08-24,2016,8,24,20:21:00,20.0,avenida,9,-58.51232237,-34.65903530,peaton-sd,peaton,NaN
92,2016-0119,1,2016-09-04,2016,9,4,06:40:00,6.0,avenida,12,-58.49659234,-34.56353635,NaN,NaN,NaN
105,2016-0135,1,2016-10-24,2016,10,24,05:00:00,5.0,avenida,14,-58.42260225,-34.56403694,peaton-sd,peaton,NaN
112,2016-0143,1,2016-11-06,2016,11,6,04:41:00,4.0,avenida,8,-58.44451316,-34.68475866,moto-sd,moto,NaN


Como se observa, los faltantes de Participantes y Víctima también coinciden con faltante en Acusado. Por otro lado, cuando se tiene uno de los datos de los participantes, pero no el segundo (por ejemplo peaton-sd), el dato faltante también está en Acusado. Dado que son 21 registros, optamos por no eliminarlos del análisis, para aprovechar la información en el resto de las columnas. La misma acción se tomará con respecto al dato faltante en las columnas HH y Hora:

In [40]:
df_hechos[df_hechos['HH'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,long,lat,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,NaN,NaN,gral paz,11,-58.52169422,-34.59471640,moto-moto,moto,moto


Antes de finalizar con este dataframe y exportarlo para su uso, vamos a analizar valores fuera de rango posible en las columnas 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH'

In [41]:
df_hechos['N_VICTIMAS'].value_counts()

N_VICTIMAS
1    674
2     19
3      1
Name: count, dtype: int64

Vemos que en la columna correspondiente a número de víctimas no hay valores fuera de rango, dado que corresponden a valores posibles y normales en accidentes de tránsito

In [42]:
df_hechos['FECHA'].min()

Timestamp('2016-01-01 00:00:00')

In [43]:
df_hechos['FECHA'].max()

Timestamp('2021-12-30 00:00:00')

Vemos que las fechas mínimas y máximas corresponden a las fechas declaradas en la fuente del dataset.

In [44]:
df_hechos['AAAA'].unique()

array([2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [45]:
df_hechos['MM'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [46]:
sorted(df_hechos['DD'].unique().tolist())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [47]:
sorted(df_hechos['HH'].unique().tolist())

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 nan]

Tanto Año, Mes, día y Franja horaria, corresponden a valores dentro del rango posible.

In [48]:
print(f'Hora máxima: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.hour.max()}')
print(f'Hora mínima: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.hour.min()}')

Hora máxima: 23.0
Hora mínima: 0.0


In [49]:
print(f'Minuto máximo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.minute.max()}')
print(f'Minuto mínimo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.minute.min()}')

Minuto máximo: 59.0
Minuto mínimo: 0.0


In [50]:
print(f'Segundo máximo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.second.max()}')
print(f'Segundo mínimo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.second.min()}')

Segundo máximo: 47.0
Segundo mínimo: 0.0


Las horas, minutos y segundos están en el rango posible.

In [51]:
df_hechos.reset_index(inplace=True, drop=True)

Exportamos el dataframe en formato CSV para luego utilizarlo en el EDA, en la herramienta de visualización seleccionada (Power BI) y en la base de datos (MySQL).

In [52]:
#df_hechos.to_csv('CleanData/homicidios_hechos.csv', index=False)
# esta celda la dejamos comentada para que no se exporte el csv cada vez que corramos el notebook completo

Cargamos ahora el dataset con datos de víctimas

In [ ]:
df_vic = pd.read_excel('Datasets/homicidios.xlsx', sheet_name='VICTIMAS')
df_vic.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [ ]:
df_vic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Veamos duplicados:

In [ ]:
df_vic.duplicated().sum()

0

Antes de verificar los nulos, pasemos todo a minúsculas y reemplacemos los posibles valores faltantes por np.nan

In [ ]:
df_vic = df_vic.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
faltantes = ['', ' ', '.', '...', 'sd']
df_vic.replace(faltantes, np.nan, inplace=True)

In [ ]:
df_vic.isna().sum()

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

La columna FECHA_FALLECIMIENTO no es de interés para nuestro análisis, por lo que es eliminada.

In [ ]:
df_vic.drop(columns='FECHA_FALLECIMIENTO', inplace=True)

Veamos los posibles valores de 'ROL'

In [ ]:
df_vic['ROL'].value_counts()

ROL
conductor               330
peaton                  267
pasajero_acompañante     80
ciclista                 29
Name: count, dtype: int64

Hagamos un merge entre el df de hechos y el de víctimas para analizar los datos faltantes

In [ ]:
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'COMUNA', 'long', 'lat',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [ ]:
df_merged = df_hechos[['ID', 'N_VICTIMAS', 'VICTIMA', 'ACUSADO']].merge(df_vic, how='left', left_on='ID', right_on='ID_hecho')
df_merged.head()

,ID,N_VICTIMAS,VICTIMA_x,ACUSADO,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA_y,SEXO,EDAD
0,2016-0001,1,moto,auto,2016-0001,2016-01-01,2016,1,1,conductor,moto,masculino,19.0
1,2016-0002,1,auto,pasajeros,2016-0002,2016-01-02,2016,1,2,conductor,auto,masculino,70.0
2,2016-0003,1,moto,auto,2016-0003,2016-01-03,2016,1,3,conductor,moto,masculino,30.0
3,2016-0004,1,moto,NaN,2016-0004,2016-01-10,2016,1,10,conductor,moto,masculino,18.0
4,2016-0005,1,moto,pasajeros,2016-0005,2016-01-21,2016,1,21,conductor,moto,masculino,29.0


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          715 non-null    object        
 1   N_VICTIMAS  715 non-null    int64         
 2   VICTIMA_x   707 non-null    object        
 3   ACUSADO     694 non-null    object        
 4   ID_hecho    715 non-null    object        
 5   FECHA       715 non-null    datetime64[ns]
 6   AAAA        715 non-null    int64         
 7   MM          715 non-null    int64         
 8   DD          715 non-null    int64         
 9   ROL         705 non-null    object        
 10  VICTIMA_y   707 non-null    object        
 11  SEXO        711 non-null    object        
 12  EDAD        664 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 72.7+ KB


Veamos si en los accidentes con más de una víctima, los datos fueron cargados correctamente

In [ ]:
df_merged[df_merged['N_VICTIMAS'] > 1]

,ID,N_VICTIMAS,VICTIMA_x,ACUSADO,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA_y,SEXO,EDAD
29,2016-0041,2,moto,cargas,2016-0041,2016-03-29,2016,3,29,conductor,moto,masculino,54.0
30,2016-0041,2,moto,cargas,2016-0041,2016-03-29,2016,3,29,pasajero_acompañante,moto,masculino,NaN
98,2016-0126,2,auto,cargas,2016-0126,2016-09-18,2016,9,18,conductor,auto,masculino,37.0
99,2016-0126,2,auto,cargas,2016-0126,2016-09-18,2016,9,18,pasajero_acompañante,auto,masculino,60.0
161,2017-0026,2,auto,objeto fijo,2017-0026,2017-02-26,2017,2,26,pasajero_acompañante,auto,femenino,23.0
162,2017-0026,2,auto,objeto fijo,2017-0026,2017-02-26,2017,2,26,conductor,auto,masculino,19.0
171,2017-0035,3,auto,objeto fijo,2017-0035,2017-03-23,2017,3,23,conductor,auto,masculino,28.0
172,2017-0035,3,auto,objeto fijo,2017-0035,2017-03-23,2017,3,23,pasajero_acompañante,auto,masculino,32.0
173,2017-0035,3,auto,objeto fijo,2017-0035,2017-03-23,2017,3,23,pasajero_acompañante,auto,masculino,30.0
174,2017-0036,2,moto,pasajeros,2017-0036,2017-03-29,2017,3,29,pasajero_acompañante,moto,femenino,NaN


Vemos que en todos los casos, están los datos completos de las víctimas de accidentes con múltiples víctimas. <br>
Veamos ahora si podemos asignar los valores de victima_x y victima_y

In [ ]:
df_merged[['VICTIMA_x', 'VICTIMA_y']][df_merged['VICTIMA_x'].isna() | df_merged['VICTIMA_y'].isna()]

,VICTIMA_x,VICTIMA_y
36,NaN,NaN
77,NaN,NaN
89,NaN,NaN
93,NaN,NaN
165,NaN,NaN
206,NaN,NaN
219,NaN,NaN
278,NaN,NaN


No podemos asignar los faltantes de un dataset en otro. Los datos de SEXO, EDAD y ROL tampoco se pueden imputar en base a otra información disponible. Por lo tanto se procede a exportar el dataset de víctimas para su posterior uso en EDA y visualización. Antes, vemos si hay valores que escapan al rango posible.

In [ ]:
df_vic.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD'],
      dtype='object')

In [ ]:
print(f'Fecha mínima: {df_vic["FECHA"].min()}')
print(f'Fecha máxima: {df_vic["FECHA"].max()}')

Fecha mínima: 2016-01-01 00:00:00
Fecha máxima: 2021-12-30 00:00:00


Las fechas mínimas y máximas están en el rango de datos declarados en la fuente del dataset.

In [ ]:
print(f"Años: {sorted(df_vic['AAAA'].unique().tolist())}")
print(f"Meses: {sorted(df_vic['MM'].unique().tolist())}")
print(f"Días: {sorted(df_vic['DD'].unique().tolist())}")

Años: [2016, 2017, 2018, 2019, 2020, 2021]
Meses: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Días: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


Todos los valores de años, meses y días corresponden a los rangos esperados

In [ ]:
df_vic['SEXO'].value_counts()

SEXO
masculino    545
femenino     166
Name: count, dtype: int64

Vemos que hay dos posibles valores en SEXO

In [ ]:
print(f"Edad mínima: {df_vic['EDAD'].min()}")
print(f"Edad máxima: {df_vic['EDAD'].max()}")

Edad mínima: 1.0
Edad máxima: 95.0


Vemos que los valores de edad están en el rango de lo posible. <br>
Ahora sí procedemos a exportar el dataframe para posterior análisis.

In [ ]:
df_vic.to_csv('CleanData/homicidios_victimas.csv', index=False, encoding= 'latin1')
# dejamos  comentada esta celda para evitar exportar el csv cada vez que se ejecuta el notebook completo

In [ ]:
df_vic.ROL.unique()

array(['conductor', 'pasajero_acompañante', 'peaton', nan, 'ciclista'],
      dtype=object)

Archivo de población según comuna, obtenido [aquí](https://www.indec.gob.ar/ftp/cuadros/poblacion/proy_1025_depto_caba.xls). <br>
En este archivo encontramos información del INDEC sobre Población estimada al 1 de julio de cada año calendario por sexo, según comuna para la Ciudad Autónoma de Buenos Aire entre los Años 2010-2025

In [ ]:
df_pob = pd.read_excel('Datasets/proy_1025_depto_caba.xls', header=4)
df_pob.head()

,Comuna,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total,3028481.0,3033639.0,3038860.0,3044076.0,3049229.0,3054267.0,3059122.0,3063728.0,3068043.0,3072029.0,3075646.0,3078836.0,3081550.0,3083770.0,3085483.0,3086680
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,243946.0,245308.0,246689.0,248069.0,249433.0,250770.0,252053.0,253271.0,254408.0,255457.0,256405.0,257235.0,257934.0,258497.0,258922.0,259205


In [ ]:
df_pob.tail()

,Comuna,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Fuente: INDEC - DGEyC del Gobierno de la Ciuda...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_pob['Comuna'].value_counts()

Comuna
Total                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    3
9                                                                                                                                                                                                                                     

Vemos que el archivo contiente 3 tablas, la primera con las estimaciones de población total, las dos siguientes con las estimaciones de población para sexo masculino y femenino. Nos interesa solo la primera, por lo que vamos a eliminar el resto. Además nos interesan sólo los años de 2016 a 2021 inclusive.

In [ ]:
df_pob.dropna(inplace=True)
df_pob = df_pob.iloc[:16,[0,7,8,9,10,11,12]]
df_pob

,Comuna,2016,2017,2018,2019,2020,2021
2,Total,3059122.0,3063728.0,3068043.0,3072029.0,3075646.0,3078836.0
4,1,252053.0,253271.0,254408.0,255457.0,256405.0,257235.0
5,2,149848.0,149720.0,149607.0,149510.0,149430.0,149371.0
6,3,192573.0,192763.0,192945.0,193115.0,193276.0,193425.0
7,4,238303.0,238809.0,239279.0,239712.0,240100.0,240437.0
8,5,186740.0,186956.0,187159.0,187348.0,187518.0,187670.0
9,6,184611.0,184846.0,185067.0,185271.0,185456.0,185620.0
10,7,240116.0,240607.0,241065.0,241484.0,241861.0,242188.0
11,8,225737.0,226649.0,227495.0,228266.0,228953.0,229541.0
12,9,170353.0,170605.0,170842.0,171062.0,171264.0,171444.0


Exportamos el archivo obtenido para utilizar en EDA y herramienta de visualización.

In [ ]:
#df_pob.to_csv('CleanData/poblacion.csv', index=False)
# dejamos esta celda comentada para evitar que se exporte el csv al correr todo el notebook

Además, necesitaremos el dataset de población en formato 'large', por lo que haremos una transformación más y exportamos:

In [ ]:
df_pob_large = df_pob.melt(id_vars='Comuna', var_name='AAAA', value_name='POBLACION').rename(columns= {'Comuna': 'COMUNA'})
df_pob_large.head()

,COMUNA,AAAA,POBLACION
0,Total,2016,3059122.0
1,1,2016,252053.0
2,2,2016,149848.0
3,3,2016,192573.0
4,4,2016,238303.0


Exportamos:

In [ ]:
#df_pob_large.to_csv('CleanData/poblacion_large.csv', index=False)
# dejamos esta celda comentada para evitar que se exporte el csv al correr todo el notebook